PRDT Data Scientist (FT) Test Task
---
Please come up with an approach for matching apps across platforms, using the attributes provided in datasets 1 and 2. You will be evaluated on (1) the quality of your code, (2) the quality of your write-up, and (3) how many matches you correctly identify in the test set. See below for more information on the deliverables we expect. 

----------

**My Approach:** 

Produce a comparative matching system that takes Dataset_1 or training-set as input and matches the gp_id with its corresponding ios_id based on fuzzy matching ratios calculated between the shared column-features, thereby, creating an output like Dataset_2 or the test set.

1. First, pre-process the data so that the index == "app_id" and all columns of interest are lower(), strip(), and punctuation replace(). Removing punctuation from publisher_name and app_name had negative effects on match capabilities, so for first procedure retain punctuation in both except for categories.

2. After pre-processing, split dataset_train into gp and ios dfs or gp_train and ios_train. These dfs will enact the procedure used to compare each value in the column field of one frame with that of the other. 

3. Using 2 simple for loops with 3 iterations, produce a match_score (app_name match), pub_score (publisher_name match), and cat_score (categories match) for each value in the column in question of gp_train with that of ios_train. The 4 ratio scores – ratio, partial ratio, token sort ratio, and token set ratio – are functions from Fuzzy Wuzzy library that calculate a Levenshtein distance / metric of similarity between string sequences. Each ratio is calculated for each pair of app_name, publisher_name, and categories, then averaged as the match_score, pub_score, and cat_score. The results – gp_id, ios_id, [app_name_gp, app_name_ios, match_score], [pub_name_gp, pub_name_ios, pub_score], [cat_name_gp, cat_name_ios, cat_score] – are appended into 3 empty lists then transformed into 3 DataFrames. 

4. Each DataFrame – match, pub, cat – contains all possible matches between the two columns, i.e. len(gp_train)*len(ios_train). After all of the combinations of matches based on app_name, publisher_name, & categories are computed, we then merge the 3 dfs into one based on gp_id & ios_id. This produces a complete matches dataframe that holds every combination of match with its corresponding scores. 

5. From here, we compute a final score – the average of the match_score, pub_score, & cat_score, which will be used to help filter the matches. Because match_score is the most important feature, we sort_values(ascending=False) with the highest match_score > pub_score > Score > cat_score dropping duplicates on gp_id & ios_id, respectively. Create protocol to label matches via if statements then sort on is_match, drop_duplicates(), then format output to match test-set. 

6. Store remaining unmatched ids for further procedures.

----------

**References:**

Pandas:
https://pandas.pydata.org/pandas-docs/stable/

LangDetect:
https://pypi.python.org/pypi/langdetect/1.0.1

Fuzzywuzzy:
https://github.com/eliracho37/fuzzywuzzy

Levensthein:
https://pypi.python.org/pypi/python-Levenshtein/0.12.0


In [160]:
### Import Libraries:

import pandas as pd
pd.options.display.max_rows = 999
from langdetect import detect
from fuzzywuzzy import fuzz

In [161]:
### Read CSV files into Pandas dfs:

dataset_1 = pd.read_csv("~/Desktop/Priori Task/Dataset_1.csv")
print("Train-set n: ",len(dataset_1))
dataset_2 = pd.read_csv("~/Desktop/Priori Task/Dataset_2.csv")
print("Test-set n: ",len(dataset_2))
dataset_3 = pd.read_csv("~/Desktop/Priori Task/Dataset_3.csv")
print("Validation set n: ",len(dataset_3))

Train-set n:  500
Test-set n:  101
Validation set n:  500


In [162]:
### 1. Preprocess data & 2. split into gp_train & ios_train:

def process(df):
    df = df.copy()
    df = df.set_index("app_id") 
    df = df.apply(lambda x: x.astype(str).str.lower())
    df = df.apply(lambda x: x.astype(str).str.strip())
    df["categories"] = df["categories"].str.replace(pat='[^a-zA-Z0-9\s]', repl=' ')
    df['lang_detected'] = df.app_name.apply(detect).astype('category')
    return df 

df = process(dataset_3)

gp_train = df[df.platform == 'gp']
print("GP-training n: ",len(gp_train))
ios_train = df[df.platform == 'ios']
print("ios-training n: ",len(ios_train))

GP-training n:  252
ios-training n:  248


In [163]:
%%time
### 3. Create function for matching procedure for app_name, publisher_name, categories: 

def find_matches(feature):

    matches = []

    for i, gp in gp_train[feature].iteritems():
        for e, ios in ios_train[feature].iteritems():
            
            ### Calculate Ratios & Score
            ratio = fuzz.ratio(gp,ios)  
            p_ratio = fuzz.partial_ratio(gp,ios)
            t_sort_ratio = fuzz.token_sort_ratio(gp,ios)
            t_set_ratio = fuzz.token_set_ratio(gp,ios)
            score = (ratio + p_ratio + t_sort_ratio + t_set_ratio) / 4
            
            ### Append results as tuples into matches list 
            matches.append((i, e, gp, ios, score))
    
    return matches 

### Cycle through tuple of features inputting find_matches(feature) then storing the results into 3 dfs:
features = ("app_name", "publisher_name", "categories")

for i in features:
    
    if "app_name" == i:
        name = pd.DataFrame(find_matches(i), columns=('gp_app_id', 'ios_app_id', 'gp_name', 'ios_name', 'match_score'))
        print("App_names matched & stored in 'name'.")
    
    elif "publisher_name" == i:
        pub = pd.DataFrame(find_matches(i), columns=('gp_app_id', 'ios_app_id', 'gp_pub', 'ios_pub', 'pub_score'))
        print("Publisher_names matched & stored in 'pub'.")
    
    else:
        cat = pd.DataFrame(find_matches(i), columns=('gp_app_id', 'ios_app_id', 'gp_cat', 'ios_cat', 'cat_score'))
        print("Categories matched & stored in 'cat'.")
    
    
### 4. Merge all 3 dfs into 1,then produce an avg. statistic for overall Score:    
cat_pub = pd.merge(cat, pub, left_on=("gp_app_id","ios_app_id"), right_on=("gp_app_id","ios_app_id"))
final = pd.merge(cat_pub, name, left_on=("gp_app_id","ios_app_id"), right_on=("gp_app_id","ios_app_id"))
final["Score"] = (final.cat_score + final.pub_score + final.match_score) / 3

### 5. Create protocol to label matches via if statements then sort on is_match, drop_duplicates(), then format output to match test-set. 
def is_match(df):
    new = df.copy()
    if df.match_score >= 50 and df.pub_score >= 50 and df.cat_score >= 50 and (df.Score > 50 or df.match_score > df.Score):
        return True
    elif df.match_score >= 61 and df.pub_score >= 0 and df.cat_score >= 0 and (df.match_score > df.Score):
        return True
    elif df.match_score >= 10 and df.pub_score >= 70 and df.cat_score >= 20 and (df.Score > 50 or df.match_score > df.Score):
        return True
    else:
        return False

final['is_match'] = final.apply(is_match, axis=1).astype('category')

final = final[final.is_match==True].sort_values(["ios_app_id","match_score","pub_score","Score","cat_score"],ascending=False).drop_duplicates("ios_app_id")
final = final[final.is_match==True].sort_values(["gp_app_id","match_score","pub_score","Score","cat_score"],ascending=False).drop_duplicates("gp_app_id")

output = final[["gp_app_id","ios_app_id","is_match"]]

output.to_csv("~/Desktop/070817_Rachovitsky_Test_Set_Predictions_Part_B.csv")

App_names matched & stored in 'name'.
Publisher_names matched & stored in 'pub'.
Categories matched & stored in 'cat'.
CPU times: user 18.3 s, sys: 56.2 ms, total: 18.3 s
Wall time: 18.3 s


In [164]:
### Accuracy:

print("Matches found: ",len(output))
print("Accuracy: ",len(output)/248*100)
output

Matches found:  146
Accuracy:  58.87096774193549


,gp_app_id,ios_app_id,is_match
43064,ztai.peimr.tmap,400872444,True
39048,za.ac.unisa.mcqApp.starter,953339420,True
11404,vp.laser.pointer.simulated,1097453810,True
25601,stellal.Sat,972372967,True
2429,ru.sputnik.browser,943916823,True
5218,qqapp.cocos2dx.math24,1019062274,True
14018,pl.speedtest.android,930377055,True
55512,nl.hqmedia.quitsmokingbuddyfree,581893877,True
18881,net.umajin.app,779381383,True
52708,net.media.store.Ro3b1,1069354702,True


In [165]:
### 6. Procedure for Remainder:

matched = []
for i in output["gp_app_id"]:
    matched.append(i)
for i in output["ios_app_id"]:
    matched.append(i)

remain = process(dataset_3[dataset_3.app_id.isin(matched)==False])

gp_remain = remain[remain.platform == 'gp']
print("gp_remain n: ",len(gp_remain))
ios_remain = remain[remain.platform == 'ios']
print("ios_remain n: ",len(ios_remain))

gp_remain n:  106
ios_remain n:  102


The Remaining: 
----
1. More pre-processing:

    a. Standardizing the categories, i.e. replacing categories w/ matching categories across platform based on dataset_2 merged with dataset_1
    
    b. Grouping gp_remain with ios_remain based on languages detected (please note, for many Chinese, detection == Korean; perhaps, having a column with the translations in english could also improve the matches... 
    
    c. Add more data such as country or location, date of release, # of users
----
N.B. Due to time restraints, I've opted for the first procedure as final answer. 

Write Up:
-----

**What were your key insights in tackling this problem?**

    Using a numerical match scoring technique via fuzzywuzzy library, I was able to create a combination of every possible match then discern which were true matches based on conditions connected to score thresholds. This made the process more numerically driven, yet the flaw of this method is that changing the parameters will in effect also change, which matches are labelled true. This would be an area to optimise more programmatically. Overall, it is important to note this method cannot handle features with completely different names or languages used. Standardizing categories, translating all non-English, and removing punctuation from app_name & publisher_name would be places to start to find the remaining matches. 


**In your opinion, are the results good enough to push to our product? Why?**

    The current result of 58.9% probably isn't the best method to put into production without dealing with protocols for the remainder. However, as a first step, this method effectively matches ids via partial string matching between gp & ios features in a clean, simple, and easy to understand way. 
    

**How would the problem change if you had to do this for 1 million apps on each platform?**

    The problem wouldn't necessarily change too greatly. Computationally it could take longer & checking for mislabelled matches could be more difficult. To combat this, the program would need an error statement for when matches are labelled on duplicate ids. With more data, it could be possible to learn patterns regarding the score thresholds, which could be used to fine tune the if statement parameters used for label matching and standardizing categories more authoritatively accross platforms.  